In [1]:
# Import library and dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from fbprophet import Prophet
plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings('ignore')

import pickle

ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.


In [2]:
raw = pd.read_csv('dss_mock_1.csv')

In [3]:
raw.describe()

,date,Nasi goreng,Mie goreng,Kwetiau goreng,Kwetiau siram,is_holiday
count,1.000000e+03,1000.0000,1000.000000,1000.000000,1000.000000,1000.000000
mean,1.571316e+09,587.6090,343.696000,331.703000,226.048000,0.188000
std,2.286453e+06,24.6876,18.659415,19.067731,15.228053,0.390908
min,1.567297e+09,496.0000,293.000000,277.000000,181.000000,0.000000
25%,1.569300e+09,571.0000,331.000000,319.000000,216.000000,0.000000
50%,1.571338e+09,588.0000,344.000000,332.000000,226.000000,0.000000
75%,1.573352e+09,604.0000,356.000000,344.000000,236.000000,0.000000
max,1.575155e+09,672.0000,401.000000,396.000000,275.000000,1.000000


In [4]:
# cleaning dataset
# convert 'epoch' date into appropriate datetime
import datetime
tf = raw.copy()
tf['date_conv'] = raw['date'].apply(lambda x : datetime.datetime.fromtimestamp(x).strftime("%Y-%m-%d"))

In [5]:
tf.drop(['date','is_holiday'], axis=1, inplace=True)

In [6]:
tf['date_conv'] = pd.DatetimeIndex(tf['date_conv'])
tf = tf.sort_values(by='date_conv').reset_index(drop=True)

In [7]:
tf.dtypes

Nasi goreng                int64
Mie goreng                 int64
Kwetiau goreng             int64
Kwetiau siram              int64
date_conv         datetime64[ns]
dtype: object

In [8]:
tf.tail()

,Nasi goreng,Mie goreng,Kwetiau goreng,Kwetiau siram,date_conv
995,600,346,329,202,2019-11-30
996,570,357,362,231,2019-11-30
997,583,355,305,216,2019-12-01
998,570,350,333,218,2019-12-01
999,565,335,312,235,2019-12-01


In [9]:
cols = list(tf.columns.values)
cols.remove('date_conv')

In [10]:
###############################################################################

In [11]:
# Model fit
for col in cols:
    subtf = tf[['date_conv',col]]
    subtf = subtf.rename(columns={'date_conv':'ds',col:'y'})
    model = Prophet(interval_width=0.95)
    model.fit(subtf)
    pickle.dump(model, open('model'+'_'+col+'.pickle','wb'))

INFO:numexpr.utils:NumExpr defaulting to 4 threads.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [12]:
# Model predict
results = []
for col in cols:
    model = pickle.load(open('model'+'_'+col+'.pickle', 'rb'))
    f_df = model.make_future_dataframe(periods=7, freq='D')
    result = model.predict(f_df)
    result_tf = result[['ds','yhat']].rename(columns={'yhat':col})
    results.append(result_tf)
tf.predict = pd.concat(results, axis=1)

In [15]:
# Cleaning predict table
tf.predict = tf.predict.iloc[:,  ~tf.predict.columns.duplicated()]
tf.predict.drop_duplicates(subset='ds', keep="first", inplace=True)

In [16]:
tf.predict = tf.predict.reset_index(drop=True).rename(columns={'ds':'date'})

In [17]:
for col in cols:
        tf.predict[col] = tf.predict[col].apply(np.floor).astype(int)

In [18]:
tf.predict.head()

,date,Nasi goreng,Mie goreng,Kwetiau goreng,Kwetiau siram
0,2019-09-01,585,344,331,227
1,2019-09-02,589,346,331,227
2,2019-09-03,586,347,333,229
3,2019-09-04,586,345,334,230
4,2019-09-05,590,345,331,227


In [19]:
tf_revised = tf.predict[(tf.predict['date']>'2019-12-01')].reset_index(drop=True)

In [20]:
tf_revised

,date,Nasi goreng,Mie goreng,Kwetiau goreng,Kwetiau siram
0,2019-12-02,587,343,326,224
1,2019-12-03,584,344,329,225
2,2019-12-04,584,342,330,226
3,2019-12-05,588,342,326,223
4,2019-12-06,580,341,328,223
5,2019-12-07,579,340,331,225
6,2019-12-08,582,341,325,222


In [30]:
today = (datetime.datetime.today() - datetime.timedelta(days=21)).strftime("%Y-%m-%d")

In [31]:
twodays = (datetime.datetime.today() + datetime.timedelta(days=1)).strftime("%Y-%m-%d")

In [32]:
tf_revised = tf.predict[(tf.predict['date']>today) & (tf.predict['date']<twodays)].reset_index(drop=True)
tf_revised

,date,food_1,food_2,food_3,food_4
0,2019-11-20,586.0,342.0,331.0,228.0
1,2019-11-21,590.0,343.0,328.0,225.0
2,2019-11-22,582.0,341.0,330.0,224.0
3,2019-11-23,582.0,340.0,332.0,227.0
4,2019-11-24,584.0,341.0,327.0,224.0
5,2019-11-25,588.0,343.0,327.0,224.0
6,2019-11-26,585.0,344.0,329.0,226.0
7,2019-11-27,585.0,342.0,330.0,227.0
8,2019-11-28,589.0,343.0,327.0,224.0
9,2019-11-29,581.0,341.0,329.0,224.0


In [23]:
print(tf_revised.to_json(orient="records"))

[{"date":1575244800000,"Nasi goreng":587,"Mie goreng":343,"Kwetiau goreng":326,"Kwetiau siram":224},{"date":1575331200000,"Nasi goreng":584,"Mie goreng":344,"Kwetiau goreng":329,"Kwetiau siram":225},{"date":1575417600000,"Nasi goreng":584,"Mie goreng":342,"Kwetiau goreng":330,"Kwetiau siram":226},{"date":1575504000000,"Nasi goreng":588,"Mie goreng":342,"Kwetiau goreng":326,"Kwetiau siram":223},{"date":1575590400000,"Nasi goreng":580,"Mie goreng":341,"Kwetiau goreng":328,"Kwetiau siram":223},{"date":1575676800000,"Nasi goreng":579,"Mie goreng":340,"Kwetiau goreng":331,"Kwetiau siram":225},{"date":1575763200000,"Nasi goreng":582,"Mie goreng":341,"Kwetiau goreng":325,"Kwetiau siram":222}]
